In [114]:
import re
import pdfplumber
import pandas as pd

def check_bboxes(word, table_bbox):
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

def process_page(page):
    tables = page.find_tables()
    table_bboxes = [i.bbox for i in tables]
    tables = [{'table': i.extract(), 'doctop': i.bbox[1]} for i in tables]
    non_table_words = [word for word in page.extract_words() if not any(
        [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
    lines = []
    for cluster in pdfplumber.utils.cluster_objects(non_table_words+tables, 'doctop', tolerance=5):
        if 'text' in cluster[0]:
            lines.append(' '.join([i['text'] for i in cluster]))
        elif 'table' in cluster[0]:
            for row in cluster[0]['table']:
                row = [item.strip().replace('\n', ' ') for item in row if item]

                if row:
                    lines.append(' | '.join(row) + '\n')

    return ' '.join(lines), tables

def clean_text(text):
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text = "\n".join(line.strip() for line in text.splitlines())
    return text.strip()

def extract_titles(text, section_depths):
    titles = []
    sorted_sections = sorted(section_depths.keys(), key=lambda s: section_depths[s], reverse=True)
    pattern = re.compile('|'.join(re.escape(s) for s in sorted_sections))
    
    matches = list(pattern.finditer(text))
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        titles.append(text[start:end])
    
    return titles

def fix_concatenated_titles(titles):
    fixed_titles = []
    title_pattern = re.compile(r'(\d+(\.\d+)*\s*)')

    for title in titles:
        if title is None:
            continue
        
        matches = list(title_pattern.finditer(title))
        last_index = 0
        current_title = ""

        for match in matches:
            start, end = match.span()
            if last_index < start:
                current_title += ' ' + title[last_index:start].strip()
            if current_title:
                fixed_titles.append(current_title.strip())
                current_title = ""
            current_title += match.group(0).strip()
            last_index = end

        if last_index < len(title):
            current_title += ' ' + title[last_index:].strip()
        
        if current_title:
            fixed_titles.append(current_title.strip().replace(' .', '.'))

    return [s for s in fixed_titles if len(s) >= 5]

def split_text_by_titles(text, titles):
    chunks = []

    for start_title, end_title in zip(titles, titles[1:]):
        start_i = text.find(start_title)
        end_i = text.find(end_title)

        if start_i == -1:
            start_i = text.find(start_title[start_title.find(' ') + 1:])
        if end_i == -1:
            end_i = text.find(end_title[end_title.find(' ') + 1:])
        
        chunk = text[start_i:end_i].strip()
        if chunk and chunk != start_title and start_i != -1 and end_i != -1:
            chunks.append({'title': start_title, 'text': chunk.replace('\n', '')})

    return chunks

def read_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        all_text = []
        titles = []
        definitions = {}

        found_table_of_content_title = False
        
        for page in pdf.pages:
            text, tables = process_page(page)
            all_text.append(clean_text(text))

            table_of_content_title = ''
    
            if not found_table_of_content_title:
                if 'СОДЕРЖАНИЕ' in text:
                    table_of_content_title = 'СОДЕРЖАНИЕ'
                elif 'Оглавление' in text:
                    table_of_content_title = 'Оглавление'
                elif 'Содержание:' in text:
                    table_of_content_title = 'Содержание:'
                elif 'Содержание' in text:
                    table_of_content_title = 'Содержание'

            if not found_table_of_content_title and table_of_content_title:
                titles = [''.join(item.strip().split('\n')) for item in re.split(r'\.{2,}', text[text.find(table_of_content_title) + len(table_of_content_title):])[:-1]]
                titles = fix_concatenated_titles([item for item in titles if item])
                found_table_of_content_title = True

            for table in tables:
                table = table['table']
                table_titles = ' '.join([item for item in table[0] + (table[1] if len(table) > 1 else []) if item]).lower()

                if ('сокращение' in table_titles and 'термин' in table_titles) or ('расшифровка' in table_titles and 'определение' in table_titles):
                    for row in table[1:]:
                        row = [item.replace('\n', '').strip() for item in row if item]

                        if len(row) < 2:
                            continue
                        
                        term = row[0]
                        definition = ' '.join(row[1:])
                        definitions[term] = definition
            
    if not titles:
        print(file_path)
        return [], [], {}
        
    joined_text = ' '.join(all_text)
    splitted_text = split_text_by_titles(joined_text[joined_text.find(titles[-1]) + len(titles[-1]):], titles)
    
    return all_text, splitted_text, definitions

In [2]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, pipeline


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to('cuda')

In [3]:
def get_embedding(text, max_length=512):
    with torch.no_grad():
        batch_dict = tokenizer([text], max_length=max_length, truncation=True, return_tensors='pt').to('cuda')
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings.cpu().numpy()

In [115]:
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import torch

inst_path = 'pdf_instructions'
instructions_paths = [f for f in listdir(inst_path) if isfile(join(inst_path, f))]

all_chunks = []
all_definitions = {}

for path in tqdm(instructions_paths):
    all_text, splitted_text, definitions = read_pdf(f'{inst_path}/{path}')

    all_definitions[path] = definitions

    for chunk in splitted_text:
        chunk_input_ids = tokenizer(chunk['text'])['input_ids']

        for slice_i in range(0, len(chunk_input_ids), 256):
            sliced_input_ids = chunk_input_ids[slice_i:slice_i + 256]

            if len(sliced_input_ids) <= 30:
                continue
            
            text = tokenizer.decode(sliced_input_ids, skip_special_tokens=True)
            embeddings = get_embedding(text)
            page_num = 0
            
            for i, page_text in enumerate(all_text):
                if text[:50] in page_text:
                    page_num = i
                    break

            for term, definition in definitions.items():
                text = text.replace(term, f'{term} ({definition})')
            
            all_chunks.append({'filename': path, 'title': chunk['title'], 'text': text, 'page_num': page_num, 'embedding': embeddings})
            torch.cuda.empty_cache()

 57%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                      | 17/30 [00:54<00:31,  2.43s/it]

pdf_instructions/Инструкция D-1C1-1.22.01 Приложение 5 Обработка по корректировке графиков.pdf


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:22<00:00,  2.75s/it]


In [116]:
[path for path in instructions_paths if not all_definitions[path]]

['Инструкция Учет внутрихозяйственных расчетов.pdf',
 'Инструкция D-1C1-1.22.01 Приложение 5 Обработка по корректировке графиков.pdf',
 'Инструкция Настройки регламентированного учета.pdf',
 'Инструкция Списание НДС на расходы.pdf',
 'D-1C1-1.05.01 Планирование работы ТС.pdf',
 'Инструкция D-1C1-1.20.02 Анализ взаиморасчетов по закупкам (1).pdf']

In [117]:
import pandas as pd

stat = pd.DataFrame([{'len': len(tokenizer(item['text'])['input_ids'])} for item in all_chunks])
stat.describe()

,len
count,1352.000000
mean,254.604290
std,77.285149
min,32.000000
25%,257.000000
50%,259.000000
75%,289.000000
max,464.000000


In [119]:
import pickle

with open('instruction_chunks.pkl', 'wb') as w:
    pickle.dump(all_chunks, w)

with open('all_definitions.pkl', 'wb') as w:
    pickle.dump(all_definitions, w)

In [9]:
questions_history = pd.read_excel('Вопросы и ответы.xlsx')
column_names = questions_history.iloc[0]
questions_history = questions_history[1:]
questions_history.columns = column_names
questions_history

,Код,Категория,Время регистрации,Рабочая группа,Краткое описание,Описание,Решение,Аналитика 1,Аналитика 2,Аналитика 3
1,C03919673,Изменение,16.08.2021 2:16:11,изм 1с erp 2.0,Без СППР. 2021 ЯФИ Заполнение ТЧ по реквизитам,в УПП была кнопка для заполнения колонки в ТЧ ...,Обращение от XXXX года неактуально.,Менеджер услуги,Другой вопрос,NaN
2,C04273748,Изменение,03.12.2021 1:09:46,изм 1с erp 2.0 запросы на развитие,Без СППР. FW: 2.0 _АО_ нет графы количества,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...,"Казначейство, взаиморасчеты",Расчеты с персоналом,Авансовые отчеты
3,C04274763,Изменение,03.12.2021 3:52:24,изм 1с erp 2.0 запросы на развитие,Без СППР. 2023 ЗИ - 1.2 ФУНКЦИОНАЛ 1С ЕРП2. ...,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Заявки на оплату
4,C04331762,Изменение,20.12.2021 1:46:04,изм 1с erp 2.0 запросы на развитие,Без СППР. Настройки для отчета по дебиторско...,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...,"Казначейство, взаиморасчеты",Взаиморасчеты,Отчеты
5,C04393042,Изменение,14.01.2022 12:45:38,изм 1с erp 2.0 запросы на развитие,Без СППР. 1.2 ФУНКЦИОНАЛ 1С ЕРП2: в дополн...,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Платежные документы
...,...,...,...,...,...,...,...,...,...,...
22362,IM23747653,инцидент,31.05.2024 3:18:22,1с erp 2.0 1-я линия,<РАОС:1С> ЮЗ ЭДО: Сделка в ЕОСДО(2024-05-31 15...,"Коллеги, во вложении скрин, по ко...",Здравствуйте. Обращение SDXXXXXXXX отозвано,NaN,NaN,NaN
22363,IM23748276,инцидент,31.05.2024 4:00:37,1с erp 2.0 1 линия - регл.учет,<Гринатом:1С> Внеоборотные активы: Основные ср...,Не выводится остаточная стоимость во вкладке О...,"Здравствуйте, Владимир Олегович! Настроила для...",Регл.учет,"Учет ОС, НМА, НИОКР, РБП",Учет ОС
22364,IM23748344,запрос на обслуживание,31.05.2024 4:03:10,1с erp 2.0 1-я линия,ФЭИ; 1С ERP 2.0; не проводится План закупок.,Добрый день. В системе XС ERP X.X создавая док...,Добрый день. а. Очищен регистр рамочных догово...,Опер.контур,Закупки,Договоры с поставщиком
22365,IM23749284,инцидент,31.05.2024 9:00:36,1с erp 2.0 1 линия - регл.учет,"<Гринатом:1С> Внеоборотные активы: РБП, ДБП(20...",Добрый день. К распределению РБП XXX создано ...,Здравствуйте. Обращение SDXXXXXXXX отозвано,Регл.учет,"Учет ОС, НМА, НИОКР, РБП",Учет РБП / ДБП


In [18]:
question_embeddings = []

for question, answer in zip(tqdm(questions_history['Описание']), questions_history['Решение']):
    if type(question) == str:
        question_embeddings.append({
            'question': question,
            'answer': answer,
            'embedding': get_embedding(question)
        })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22366/22366 [04:17<00:00, 86.92it/s]


In [19]:
with open('question_embeddings.pkl', 'wb') as w:
    pickle.dump(question_embeddings, w)